# Chapter 01 - Introduction

In [ ]:
# Typical imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

In [ ]:
# Incorporate the custom `matplotlib` styles
import json
from cycler import cycler
s = json.load(open('./styles/bmh_matplotlibrc.json'))
s['axes.prop_cycle'] = cycler('color', s['axes.prop_cycle']['color'])
matplotlib.rcParams.update(s)

Plot a sequence of posterior probabilities updated with increasing amounts of data (coin flips)

In [ ]:
# Set width and height of figure in inches
plt.figure(figsize=(11, 9), dpi=300)

In [ ]:
# The prior is drawn from a beta distribution
# (the conjugate prior of the Bernoulli)
dist = stats.beta
# Sampled linearly 100 times
x = np.linspace(0, 1, 100)

# The data is drawn from a Bernoulli distribution
n_trials = [0, 1, 2, 3, 4, 5, 8, 15, 50, 500]
data = stats.bernoulli.rvs(0.5, size=n_trials[- 1])

In [ ]:
for i, N in enumerate(n_trials):
    # draw on `i`th axes of `len(n_trials) // 2` rows by 2 columns
    ax = plt.subplot(len(n_trials) // 2, 2, i + 1)
    plt.xlabel('$p$, probability of heads') if i in [0, len(n_trials) - 1] else None
    plt.setp(ax.get_yticklabels(), visible=False)
    heads = data[:N].sum()  # Calculate the number of successes (heads)
    # Our initial prior is a uniform beta (a=1, b=1).
    # Add `heads` to $\alpha$; tails (`N - heads`) to $\beta$
    y = dist.pdf(x, 1 + heads, 1 + N - heads)
    plt.plot(x, y, label=f'Observe {N} tosses,\n {heads} heads')
    plt.fill_between(x, 0, y, color='#348abd', alpha=0.4)
    plt.vlines(0.5, 0, 4, color='k', linestyles='--', lw=1)

    legend = plt.legend()
    legend.get_frame().set_alpha(0.4)
    plt.autoscale()

# Title of all plots
plt.suptitle('Bayesian updating of posterior probabilities', y=1.02, fontsize=14)
plt.show()